In [22]:
import kmapper as km
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN

In [23]:
# crear DataFrame con 3 columnas y 500 filas
df = pd.DataFrame(np.random.randint(1, 101, size=(500, 3)), columns=['Parcial 1', 'Parcial 2', 'Parcial 3'])

# agregar cuarta columna con promedio de toda la fila
df['Promedio'] = df.mean(axis=1)
df["Promedio"] = df["Promedio"].astype(int)

In [24]:
# mostrar primeras 3 filas del DataFrame
print(df.head(3))

   Parcial 1  Parcial 2  Parcial 3  Promedio
0         79          8         15        34
1         42         18         92        50
2         22         77         76        58


In [25]:
df

,Parcial 1,Parcial 2,Parcial 3,Promedio
0,79,8,15,34
1,42,18,92,50
2,22,77,76,58
3,62,82,11,51
4,37,38,19,31
...,...,...,...,...
495,20,45,4,23
496,94,83,99,92
497,68,81,23,57
498,7,85,72,54


In [26]:
cubierta = [(0, 40), (30, 70), (60, 100)]
cubierta

[(0, 40), (30, 70), (60, 100)]

In [27]:
def metrica(v, w):
    var = 0
    for i in range(0, 3):
        if var < abs(v[i] - w[i]):
            var = abs(v[i] - w[i])
    return var

In [28]:
df.iloc[1]

Parcial 1    42
Parcial 2    18
Parcial 3    92
Promedio     50
Name: 1, dtype: int64

In [29]:
df.iloc[0]

Parcial 1    79
Parcial 2     8
Parcial 3    15
Promedio     34
Name: 0, dtype: int64

In [30]:
metrica(df.iloc[1], df.iloc[0])

77

In [31]:
X_array = df[['Parcial 1', 'Parcial 2', 'Parcial 3']].to_numpy()

In [32]:
X_array

array([[79,  8, 15],
       [42, 18, 92],
       [22, 77, 76],
       ...,
       [68, 81, 23],
       [ 7, 85, 72],
       [22, 89, 24]])

In [33]:
dbscan = DBSCAN(eps=10, min_samples=5, metric=metrica).fit(X_array)

In [34]:
mapper = km.KeplerMapper(verbose=2)

lens2 = mapper.fit_transform(X_array, projection="x[1]")

lens1 = mapper.fit_transform(X_array, projection="l2norm")

lens = np.c_[lens1, lens2]

KeplerMapper(verbose=2)
..Composing projection pipeline of length 1:
	Projections: x[1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (500, 3)

..Projecting data using: x[1]

..Scaling with: MinMaxScaler()

..Composing projection pipeline of length 1:
	Projections: l2norm
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (500, 3)

..Projecting data using: l2norm

..Scaling with: MinMaxScaler()



In [35]:
graph = mapper.map(
    X_array,
    clusterer=dbscan,
    cover=km.Cover(n_cubes=3, perc_overlap=0.4),
)

informacion = mapper.data_from_cluster_id('cube2_cluster0', graph, X_array)

print(informacion)

Mapping on data shaped (500, 3) using lens shaped (500, 3)

Minimal points in hypercube before clustering: 5
Creating 27 hypercubes.
   > Found 3 clusters in hypercube 0.
   > Found 3 clusters in hypercube 1.
   > Found 3 clusters in hypercube 2.
   > Found 2 clusters in hypercube 3.
   > Found 2 clusters in hypercube 4.
   > Found 3 clusters in hypercube 5.
   > Found 1 clusters in hypercube 6.
   > Found 1 clusters in hypercube 7.
   > Found 3 clusters in hypercube 8.
   > Found 4 clusters in hypercube 9.
   > Found 2 clusters in hypercube 10.
   > Found 2 clusters in hypercube 11.
   > Found 1 clusters in hypercube 12.
   > Found 1 clusters in hypercube 13.
   > Found 2 clusters in hypercube 14.
   > Found 4 clusters in hypercube 15.
   > Found 5 clusters in hypercube 16.
   > Found 5 clusters in hypercube 17.
   > Found 2 clusters in hypercube 18.
   > Found 2 clusters in hypercube 19.
   > Found 2 clusters in hypercube 20.
   > Found 3 clusters in hypercube 21.
   > Found 4 cluste

In [38]:
df[["Promedio"]].values

array([[34],
       [50],
       [58],
       [51],
       [31],
       [50],
       [67],
       [58],
       [33],
       [72],
       [43],
       [43],
       [72],
       [47],
       [32],
       [42],
       [43],
       [17],
       [64],
       [51],
       [28],
       [28],
       [58],
       [30],
       [38],
       [27],
       [35],
       [45],
       [20],
       [45],
       [75],
       [58],
       [64],
       [50],
       [68],
       [31],
       [40],
       [58],
       [57],
       [50],
       [41],
       [50],
       [44],
       [66],
       [60],
       [51],
       [74],
       [26],
       [71],
       [40],
       [28],
       [71],
       [44],
       [69],
       [58],
       [69],
       [66],
       [51],
       [61],
       [48],
       [55],
       [65],
       [17],
       [48],
       [32],
       [56],
       [42],
       [33],
       [57],
       [76],
       [36],
       [38],
       [57],
       [52],
       [40],
       [32],
       [64],

In [40]:
mapper.visualize(
    graph,
    path_html="circulomapper.html",
    title="Circulo Mapper",
    color_values=lens,
    colorscale=None,
    nbins=8,
    color_function_name=['Proyeccion en el eje x', 'Norma L2', 'kjkj', 'dlkdl'],
    node_color_function=['mean', 'std', 'median', 'max'],
    X_names=df[["Promedio"]].values
)

Wrote visualization to: circulomapper.html


'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Circulo Mapper | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  font-size: 13px;\n  font-weig

In [ ]:
km.draw_matplotlib(graph)

plt.show()

In [41]:
df.iloc[68]

Parcial 1    64
Parcial 2    29
Parcial 3    79
Promedio     57
Name: 68, dtype: int64

In [43]:
df.iloc[444]

Parcial 1    46
Parcial 2    39
Parcial 3    72
Promedio     52
Name: 444, dtype: int64